# _This Project contains 3 modules under the name "The_document_helper" namely :-


# 1) _Module : CV Assistant.
# 2) _Module : Pdf Merger.
# 3) _Module : Pdf to text Converter.

# ........................................................................................................................

# 1) Module : CV Assistant

## _Description : Analysing pdf and identifying the maximum occuring keywords , then                                         using the keywords to check for curse words and giving sugestions to                                       replace them 

##                       : ( internet connectivity required )   

##                       : IT  gives meaning and synonyms of the curse words ,suggestive words                                    for  replacing the curse words.

## _Usage : Can be used as an  invigilator to your CV OR RESUME ( checking for curse                                words and  identifying the frequency of keywords you used and then suggesting                      words against curse words )

**_STEP  1     - using tkinter open dialog for selecting your resume or any other document _**

In [1]:
import tkinter
from tkinter import *
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import showerror

class MyFrame(Frame):
    def __init__(self):
        Frame.__init__(self)
        self.master.title("PDF ANALYSIS FOR CURSE WORDS")
        self.master.rowconfigure(5, weight=1)
        self.master.columnconfigure(5, weight=1)
        self.grid(sticky=W+E+N+S)

        self.button = Button(self, text="Choose  your pdf ", command=self.load_file, width=100)
        self.button.grid(row=1, column=0, sticky=W)

    def load_file(self):
        fname = tkinter.filedialog.askopenfilename(filetypes=(("Pdf files", "*.pdf") 
                                          # ,("HTML files", "*.html;*.htm"),
                                           ,("All files", "*.*") ))#used for multiple files selection
        
        
        if fname:
            try:
                print("Success")
            except:                     # <- naked except is a bad idea
                showerror("Open Source File", "Failed to read file\n'%s'" % fname)
            return fname
        
a=MyFrame().load_file()
print(a)

Success
/home/arjun/13_april/project file and cv/updated cv/ARJUN RESUME CSe.pdf


**_STEP  2     - identifying frequency of keywords used  _**`

In [2]:
import PyPDF2 
import textract
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords')

pdfFileObj = open(a,'rb')
#The pdfReader variable is a readable object that will be parsed
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#discerning the number of pages will allow us to parse through all #the pages
num_pages = pdfReader.numPages
count = 0
text = ""
#The while loop will read each page
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
#This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
if text != "":
   text = text
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
else:
   text = textract.process(fileurl, method='tesseract', language='eng')
# Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
# Now, we will clean our text variable, and return it as a list of keywords.

#The word_tokenize() function will break our text phrases into #individual words
tokens = word_tokenize(text)
#we'll create a new list which contains punctuation we wish to clean
punctuations = ['(',')',';',':','[',']',',']
#We initialize the stopwords variable which is a list of words like #"The", "I", "and", etc. that don't hold much value as keywords
stop_words = stopwords.words('english')
#We create a list comprehension which only returns a list of words #that are NOT IN stop_words and NOT IN punctuations.
keywords = [word for word in tokens if not word in stop_words and  not word in punctuations]

frequency_list = {}
for word in keywords:
    if word not in frequency_list:
        frequency_list[word] = 1
    else:
        frequency_list[word] += 1
#print(frequency_list)
print([(k,v) for v,k in sorted([(v,k) for k,v in frequency_list.items()],reverse=True)])


[('.', 35), ('Title', 12), ('Description', 10), ('using', 5), ('http', 5), ('system', 4), ('online', 4), ('examination', 4), ('developed', 4), ('calling', 4), ('Verbose', 4), ('Python', 4), ('Paper', 4), ('Organization', 4), ('Guess', 4), ('Generator', 4), ('Duration', 4), ('Android', 4), ('application', 3), ('Word', 3), ('Lucknow', 3), ('//demo.mediatrenz.com/citystyle/', 3), ('-', 3), ('year', 2), ('work', 2), ('store', 2), ('shopping', 2), ('question', 2), ('probability', 2), ('previous', 2), ('press', 2), ('papers', 2), ('paper', 2), ('organization', 2), ('months', 2), ('messaging', 2), ('maximum', 2), ('knowledge', 2), ('generates', 2), ('free', 2), ('features', 2), ('facilities', 2), ('digipodium', 2), ('develope', 2), ('data', 2), ('cost', 2), ('chatting', 2), ('built', 2), ('best', 2), ('based', 2), ('automatically', 2), ('applications', 2), ('app', 2), ('analyses', 2), ('Used', 2), ('URL', 2), ('Technical', 2), ('System', 2), ('Software', 2), ('Real-time', 2), ('Questions', 2)

**_STEP  2     - Performing Profanity Check on identified frequency keywords used  _**`

In [3]:
#import urllib.request
import requests
lis=[]

#below is a function for progress bar
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

#below is a function for finding the curse words       
def check_content_profanity(contents):
    print(' Checking for Curse words , Please wait....\n')
    flag=0
    count=0
    for x in log_progress(contents,every=1):
        output = requests.get('http://www.wdylike.appspot.com/?q='+x).text
        #connection = urllib.request.urlopen('http://www.wdylike.appspot.com/?q='+x)
        #output=connection.read()
        #print(x+'  '+output)
        if(output=='true'):
            lis.append(x)
            count+=1
            print(x+"\n is a curse word ")
            flag=1
    if(flag==1):
            print('\n The document has ' + str(count) +' curse words and must be reconsidered for content ')
    else:
        print('\n The document has no curse words and is safe for usage ')  
    #connection.close()
    
check_content_profanity(frequency_list.keys())



 Checking for Curse words , Please wait....



VBox(children=(HTML(value=''), IntProgress(value=0, max=265)))

PASSING
 is a curse word 
Assistant
 is a curse word 

 The document has 2 curse words and must be reconsidered for content 


**_STEP   3   - Finding meaning and synonyms of the identified curse words  _**`

In [4]:
from nltk.corpus import wordnet
  
def find_synonym(x):
    synonyms = []
    for syn in wordnet.synsets(x):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return (set(synonyms))

for x in lis:   
    syn = wordnet.synsets(x)
    print('\n'+x+' MEANS \n'+syn[0].definition()+'\n')
    print('SYNONYMS of '+x+' are \n')
    print(find_synonym(x))


PASSING MEANS 
(American football) a play that involves one player throwing the ball to a teammate

SYNONYMS of PASSING are 

{'fall', 'kick_the_bucket', 'ephemeral', 'blow_over', 'give-up_the_ghost', 'go_across', 'reach', 'turn_over', 'go_on', 'transcend', 'drop_dead', 'authorise', 'lead', 'fall_out', 'exceed', 'perish', 'qualifying', 'pass_off', 'pass', 'authorize', 'release', 'go', 'make_pass', 'overtake', 'return', 'passing_play', 'draw', 'clear', 'pass_along', 'passing', 'short-lived', 'evanesce', 'expiration', 'go_along', 'passing_game', 'casual', 'overstep', 'loss', 'sink', 'give', 'overhaul', 'come_about', 'snuff_it', 'legislate', 'go_by', 'pop_off', 'communicate', 'perfunctory', 'slip_by', 'fleet', 'put_across', 'cursory', 'run', 'guide', 'fade', 'make_it', 'go_past', 'travel_by', 'buy_the_farm', 'lapse', 'going', 'egest', 'occur', 'expire', 'super', 'exit', 'surpass', 'eliminate', 'transitory', 'take_place', 'top', "cash_in_one's_chips", 'passage', 'happen', 'croak', 'go_thr

**_STEP   4   - Finding suggestions to replace the identified curse words  _**`

In [5]:
import urllib.request
def suggestion(words):
    lis1=[]
    #print(' Checking for Suggestions , Please wait....\n')
    for x in log_progress(words,every=1):
        connection = urllib.request.urlopen('http://www.wdylike.appspot.com/?q='+x)
        output=connection.read()
        if(output==b'false'):
            lis1.append(x)
    connection.close()
    print(lis1)
    
    
for x in lis:   
    print('\n SUGGESTIONS TO REPLACE '+x+' are ...\n')
    y=find_synonym(x)
    suggestion(y)


 SUGGESTIONS TO REPLACE PASSING are ...



VBox(children=(HTML(value=''), IntProgress(value=0, max=99)))

['fall', 'kick_the_bucket', 'ephemeral', 'blow_over', 'give-up_the_ghost', 'go_across', 'reach', 'turn_over', 'go_on', 'transcend', 'drop_dead', 'authorise', 'lead', 'fall_out', 'exceed', 'perish', 'qualifying', 'authorize', 'release', 'go', 'overtake', 'return', 'draw', 'clear', 'short-lived', 'evanesce', 'expiration', 'go_along', 'casual', 'overstep', 'loss', 'sink', 'give', 'overhaul', 'come_about', 'snuff_it', 'legislate', 'go_by', 'pop_off', 'communicate', 'perfunctory', 'slip_by', 'fleet', 'put_across', 'cursory', 'run', 'guide', 'fade', 'make_it', 'go_past', 'travel_by', 'buy_the_farm', 'lapse', 'going', 'egest', 'occur', 'expire', 'super', 'exit', 'eliminate', 'transitory', 'take_place', 'top', "cash_in_one's_chips", 'happen', 'croak', 'go_through', 'exceedingly', 'hand', 'elapse', 'overtaking', 'choke', 'decease', 'extend', 'extremely', 'excrete', 'slip_away', 'spend', 'devolve', 'slide_by', 'hap', 'departure', 'fugacious', 'transient', 'die', 'conk', 'glide_by']

 SUGGESTIONS

VBox(children=(HTML(value=''), IntProgress(value=0, max=5)))

['help', 'supporter', 'helper', 'adjunct']


# ........................................................................................................................

# Restarting kernel after end of module

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# 2) Module : pdf merger

## _Description : Selecting multiple pdf files with dialog and merging selected files into a                                     single pdf named      "combined_result.pdf"_

## _Usage : for example - can be used to merge all exam papers and specific study material                        into a  single common pdf

**_STEP  1     - defining tkinter open dialog for multiple selection and show selected files as a list_**

In [2]:
import tkinter
from tkinter import *
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import showerror

class MyFrame(Frame):
    def __init__(self):
        Frame.__init__(self)
        self.master.title("PDF MERGER")
        self.master.rowconfigure(5, weight=1)
        self.master.columnconfigure(5, weight=1)
        self.grid(sticky=W+E+N+S)

        self.button = Button(self, text="Choose previous year pdf papers", command=self.load_file, width=100)
        self.button.grid(row=1, column=0, sticky=W)

    def load_file(self):
        fname = tkinter.filedialog.askopenfilenames(filetypes=(("Pdf files", "*.pdf")
                                           #,("HTML files", "*.html;*.htm"),
                                           ,("All files", "*.*") ))#used for multiple files selection
        root=tkinter.Tk()
        files=root.tk.splitlist(fname)
        pdf=list(files)
        
        if fname:
            try:
                print("Success")
            except:                     # <- naked except is a bad idea
                showerror("Open Source File", "Failed to read file\n'%s'" % fname)
            return pdf

**_STEP-2   CALLING tkinter open dialog and GETTING list from the class defined in the variable a , printing the files selected with their path , and also printing length of list i.e no. of files selected_**

In [3]:
a=MyFrame().load_file()
print(a)
print(len(a))


Success
['/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/btech-cs-7-sem-pattern-recognition-ecs-074-2016.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/btech-cs-8-sem-pattern-recognition-ncs-080-2017.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/btech-it-7-sem-pattern-recognition-ecs-074-2016.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/ecs-074 pattern recognition 2011-12.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/ecs-074 pattern recognition 2012-13.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/ecs-074 pattern recognition 2013-14.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern Recognition/ecs-074 pattern recognition 2014-15.pdf', '/home/arjun/13_april/3 march pic_paper/8th sem b.tech/NCS-080  Pattern R

**_STEP-3       LIST containing path of files selected stored in a variable b to access later on_**

In [4]:
b=a
print(type(b))

<class 'list'>


**_STEP-4   MERGING OF PDF and saving pdf in location of current notebook_**

In [5]:
from PyPDF2 import PdfFileMerger, PdfFileReader
merger = PdfFileMerger()
for filename in b:
    merger.append(PdfFileReader(filename, 'rb'))

merger.write("combined_result.pdf")


# ........................................................................................................................

# Restarting kernel after end of module

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# 3) Module : pdf to text converter

## _Description : Selecting  pdf file with dialog and converting it to equivalent text file

## _Usage :The text file created Can be used to perform easy data analysis on the desired                         pdf file

**_STEP  1     - using tkinter open dialog for selecting pdf which is to be converted to text _**

In [1]:
import tkinter
from tkinter import *
from tkinter.filedialog import askopenfilename
from tkinter.messagebox import showerror

class MyFrame(Frame):
    def __init__(self):
        Frame.__init__(self)
        self.master.title("PDF ANALYSIS FOR CURSE WORDS")
        self.master.rowconfigure(5, weight=1)
        self.master.columnconfigure(5, weight=1)
        self.grid(sticky=W+E+N+S)

        self.button = Button(self, text="Choose  your pdf ", command=self.load_file, width=100)
        self.button.grid(row=1, column=0, sticky=W)

    def load_file(self):
        fname = tkinter.filedialog.askopenfilename(filetypes=(("Pdf files", "*.pdf") 
                                          # ,("HTML files", "*.html;*.htm"),
                                           ,("All files", "*.*") ))#used for multiple files selection
        
        
        if fname:
            try:
                print("Success")
            except:                     # <- naked except is a bad idea
                showerror("Open Source File", "Failed to read file\n'%s'" % fname)
            return fname
        
a=MyFrame().load_file()
print(a)

Success
/home/arjun/13_april/pdf_modules/combined_result.pdf


**_STEP  2     - converting selected pdf file to text file and saving it in location of current notebook with name                   "pdf_to_text.txt" _**

In [2]:
import PyPDF2
pdfFileObj = open(a,'rb')     #'rb' for read binary mode
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
total_pages=pdfReader.numPages
f= open("pdf_to_text.txt","w+")
print(range(total_pages))
#pdfs=[]
for i in range(total_pages):
    pageObj = pdfReader.getPage(i)          #'9' is the page number
    data=pageObj.extractText()
    f.write(data)
    print(data)
    #print(pdfs[i])
f.close() 

range(0, 32)
\7Printed Pages: 3{Following Paper ID and Roll }tlo. to he filtred in yourAmswer Booli)Roll No.B.Tech.(sEM. VItr) THEORY' EXAMThTATION, 2t}15-1 6PATTERITRECOGiTTfiON[Time:3 hoursJ[MaximumMarks:100Section*AQ. I Attexnpt all parts. AII parts •arry equal marks. WriteanslYer of each part in short.{2" 1 0:20)(a) Srrite the differ:snce between classification andclustering.(b) It-A, B, C are three mutual exclusive and exhaustiveevents and P{B)_-Il3 P{A), P(tl)-112 P{A), findP(A), P(B) andP(C).(1)ECS-074ECS-074www.aktuonline.comwww.aktuonline.comwww.aktuonline.com
TiHfxn*i MI(c) liryo cards are clrar,vn ficrn a full packet of 52 cards,the first card being returned to the packet befarethe seconcl is drawn. trrind the probability that these'trvo cards are of the safile suit.(d) What is the difference hetrryeen pararnetic and rlonparffn etric pattreffi recogintion m ethods?(e) \fuhat is the prohability of obtaining 9, 10, end 1 1points r,vith 3 dice?(0 l{ow do we evaluate the perform

2. Attempt any two parts : (10x2:20)(a) What is Bay's Theorem ? Explain. Also discuss Bay'sClassifier using some example in detail.(b) What is a discriminant function ? Discuss it in detail. In atwo class problem, the likelihood ratio is given as follows :p(xlC,) / p(xlC,)Write the discriminant function in terms of the likelihoodratio.(c) Describe the following with suitable example :(D NormalDensityFunction(iD UtilityTheory.3. Attempt any two parts : (10x2=20)(a) What do you mean by dimension reduction ? DiscussPrincipal Component Analysis (PCA) algorithm fordimension reduction.(b) Write short notes on the following :(D Maximum-Likelihoodestimation.(il) Expectation-maximization(c) Write a short note on Hidden Markov Model (HMM).4. Attempt any two parts : (10x2:20)(a) What do you meanby fuzzy decision making ? Also discussthe fuzzy classifi cation using suitable example.(b) Write an algorithm for K-Nearest neighbor estimation.Explain.ECS0T4tKlH-26642http://www.UPTUonline.comuptuonline.

c) What is statistical pattern recognition ? Explain.d) What is K-Near Neighbours classifier ?e) Consider the two class problem class 1 and class 2.Test pattern is P. Five Nearest Neighbours patterns totest pattern P is Xl, ){..Z, ){3, X4 and X5. Xl andX4 belong to Class I and X2, X3 and X5 belong toClass 2. Distances from p, d(Xl, p) : l,d(XT, P) : Z, d(X3, p) : 2.5, d(X4, p) : 4 andd(X5, P) : 5. Classify the test pattern p using modifiedk-Nearest Neighbour algorithm.0 What is learning and adaption ? ExplainAttempt any four of the following :(4x5=20)a) Explain Bayesian decision theory. Explain two categoryclassifications.b) tet blue, green, and red be three classes of objectswith prior probabilities given by p(blue) : ll4,P(green) : l/2, P(red) : l/4. Let there be three typesof objects : pencils, pens, and paper. Let theclass-conditional probabilities of these objects be1107s41I Contd...http://www.UPTUonline.comuptuonline.comuptuonline.comuptuonline.com
l*-IP(penc,{llgreen): 1/3iP(pen

In [ ]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# ........................................................................................................................